# Particle Set


Simulate trajectories of a particle cloud in a two-dimensional flow field.
A doubly-periodic domain and randomly-generated flow fields are initially used.
For additional documentation e.g. see :
[1](https://JuliaClimate.github.io/IndividualDisplacements.jl/dev/),
[2](https://JuliaClimate.github.io/MeshArrays.jl/dev/),
[3](https://docs.juliadiffeq.org/latest/solvers/ode_solve.html),
[4](https://en.wikipedia.org/wiki/Displacement_(vector))

Exercises:
- change the initial distribution of partices
- increase the duration of the trajectories simulation
- treat the non-periodic domain case by padding `u,v` with zeros
- replace `u,v` with your own two-dimensional flow fields

![particles in random flow](https://github.com/JuliaClimate/IndividualDisplacements.jl/raw/master/examples/figs/RandomFlow.gif)

## 1. Import Software

In [1]:
using IndividualDisplacements, DataFrames
p=dirname(pathof(IndividualDisplacements))
include(joinpath(p,"../examples/helper_functions.jl"))

Main.##256.isosurface

## 2. Flow Field

The `u,v` arrays below can be replaced with any other pair provided by the user.
A couple of important considerations, however:

- `u,v` are staggered on a C-grid; by `-0.5` grid point in direction `1` for `u` (`2` for `v`)
 from the grid cell center (0.5,0.5)
- `u,v` here derive from streamfunction `ϕ`, defined at the corner point, which ensures that
 the resulting `u,v` is non-divergent, purely rotational, over the C-grid domain.
In brief:

```
u=-(circshift(ϕ, (0,-1))-ϕ)
v=(circshift(ϕ, (-1,0))-ϕ)
```

In [2]:
u,v,ϕ=setup_random_flow()

([0.005736387159117694 0.005385031076733757 … 0.0050915460238911345 0.0052722459422204215; 0.005558563833222868 0.005101620913693546 … 0.005962227987957654 0.0058308128988442745; … ; 0.003545757628188567 0.005556096780027553 … -0.002403831254719907 -8.752982889700944e-5; 0.004606131542026867 0.0058587278970432255 … 0.0017399106107752194 0.003433834511864395], [-0.0015633827727080732 -0.0013855594468132476 … -0.000134133852017701 -0.0010048158160842202; 0.0051213453955241475 0.006244092717557992 … 0.001960801966553155 0.0037098750270188954; … ; -0.012124702816487462 -0.013185076730325762 … -0.0044595966102309315 -0.008603338475726058; -0.007575037077769027 -0.008705292694859854 … -0.0023849902342970852 -0.005736625647413], [0.05572749834798237 0.04999111118886468 … 0.06609129031409393 0.060999744290202794; 0.0541641155752743 0.04860555174205143 … 0.06595715646207623 0.059994928474118574; … ; 0.07542723824223886 0.0718814806140503 … 0.07293587715862195 0.07533970841334185; 0.063302535425

If user were to start with collocated velocity (`uC,vC` at the grid cell center) then
one can easily obtain the staggered velocity (`u,v`) as follows. These may contain both
[rotational and divergent](https://en.wikipedia.org/wiki/Helmholtz_decomposition) components.

```
u=0.5*(circshift(uC, (0,1))+uC)
v=0.5*(circshift(vC, (1,0))+vC)
```

## 3. Initialize Individuals

For example, we can initialize 100 particles within a central subdomain as follows.

In [3]:
np,nq=size(u)
x=np*(0. .+ 1.0*rand(1000))
y=nq*(0. .+ 1.0*rand(1000));

The `setup_point_cloud` function then wraps everything in the `Individuals` data structure.

In [4]:
𝐼=setup_point_cloud(u,v,X=x,Y=y)
𝐼.𝑃.𝑇[2]=10.
𝐼.🔴

,ID,x,y,t
,Int64,Float64,Float64,Float64


## 3. Compute Trajectories

The time period is `𝐼.𝑃.𝑇` by default, unless `∫!(𝐼,𝑇)` is called instead.

In [5]:
∫!(𝐼)

1×1000 Array{Array{Float64,1},2}:
 [7.17818, 12.0352, 1.0]  [9.29643, 4.29416, 1.0]  …  [7.90686, 2.64335, 1.0]

## 4. Plot Results

For example, generate a simple animation:

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*